# Peak fitting tool 

## Summary

This is an online tool to fit a Voigt peak to a x,y dataset. 

## How to use

Upload a csv file (";" delimiter, first column "x" second column "y") using the upload button. Adjust parameters using widgets. Once parameters are close to fit, click to optimize button to perform curve fitting. 

In [5]:
import io
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

from scipy.optimize import curve_fit
%matplotlib widget

In [6]:
from scipy.special import wofz

def voigt(x, A, x0, sigma, gamma):
    """Voigt peak 

    x ... x values, numpy.array
    A ... integral, float
    x0 ... center, float
    sigma ... width gaussian,float
    gamma ... width lorenzian, float"""
    z = ((x-x0) + 1j *gamma)/(np.sqrt(2) * sigma)
    return A*np.real(wofz(z))/np.sqrt(2*np.pi)/sigma

In [7]:
class InteractiveFitting:

    def __init__(self, peak_shape, peak_args, x_data=None, y_data=None):
        """
        peak_shape : function, 
                     first arg x, other args see peak args
                     returns y data

        peak_args : list of tuples,
                    args for peak_shape
                    first element of tuple: arg name (str)
                    second element: lower limit
                    third element: upper limit

        """
        self._peak_shape = peak_shape
        self._peak_args = peak_args 
        if x_data is None or y_data is None:
            x_data = np.array([])
            y_data = np.array([])

        self.x_data = x_data
        self.y_data = y_data

        self.setup_plot()
        self.setup_widgets()
        self.setup_button()
        self.setup_upload()
        self.setup_layout()
        
        
        self.update_fit()
        



    def get_params(self):
        params = []
        for name,_, _ in self._peak_args:
            params.append(self.widgets[name].value)
        return params

    def set_params(self, params):
        for param, (name,_,_) in zip(params, self._peak_args):
            self.widgets[name].value = param
    
    def update_fit(self, *args):
        fit_x = self.x_data
        params = self.get_params()
        fit_y = self._peak_shape(fit_x, *params)
        self.l_fit.set_xdata(fit_x)
        self.l_fit.set_ydata(fit_y)



    def setup_plot(self):
        with plt.ioff():
            fig, ax = plt.subplots()
            l_fit = ax.plot([],[], label="fit")[0]
            l_raw = ax.plot(self.x_data, self.y_data, linestyle="none",marker="x",  label="raw data")[0]
            ax.legend();
            self.fig = fig
            self.l_raw = l_raw
            self.l_fit = l_fit
            self.fig.canvas.header_visible = False

    def optimize(self, x_data, y_data, p0):
        ret = curve_fit(self._peak_shape, x_data, y_data, p0)
        return ret[0]

    def upload_listener(self, event):
        new_file = event["new"][-1] # might be more than one, only use one
        xdata, ydata = np.loadtxt(io.BytesIO(new_file["content"]), delimiter=";", unpack=True)

        # could potentially be moved to setter/getter
        self.x_data = xdata
        self.y_data = ydata
        self.l_raw.set_xdata(xdata)
        self.l_raw.set_ydata(ydata)

        self.fig.axes[0].relim()
        self.fig.axes[0].autoscale()
        self.fig.canvas.draw_idle()

    def setup_upload(self):
        self.file_uploader = widgets.FileUpload(accept=".csv", 
                                  layout=widgets.Layout(width='auto', height='auto'))
        self.file_uploader.observe(self.upload_listener, names=["value"])

    def on_click_optimize(self, *args):
        p0 = self.get_params()
        p1 = self.optimize(self.x_data, self.y_data, p0)
        self.set_params(p1)

    def setup_button(self):
        self.btn_optimize = widgets.Button(description="Optimize!",
                                  layout=widgets.Layout(width='auto', height='auto'))
        self.btn_optimize.on_click(self.on_click_optimize)

    def setup_widgets(self):
        self.widgets = {name: widgets.FloatSlider( (vmin + vmax)/2, 
                                                  min = vmin, max=vmax, step = (vmax-vmin)/1000, 
                                                  description=name)
                                                   for name, vmin, vmax in self._peak_args}
        self.widget_layout = widgets.VBox([self.widgets[name] 
                                           for name, vmin, vmax in self._peak_args])
        for widget in self.widget_layout.children:
            widget.observe(self.update_fit, names=["value"])

    def setup_layout(self):
        self.layout = widgets.AppLayout(left_sidebar=self.fig.canvas, 
                                        right_sidebar=self.widget_layout, justify_items='center',
                                        footer = widgets.HBox([self.file_uploader, self.btn_optimize]),
                                        width="75%",
                                        align_items='center')

        

In [8]:
ifit = InteractiveFitting(voigt, [("A", 0, 15),("x0", 0, 10),("sigma", 0.00000001,15),  ("gamma",0, 15)])
ifit.layout

AppLayout(children=(HBox(children=(FileUpload(value=(), accept='.csv', description='Upload', layout=Layout(hei…